In [23]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from joblib import Parallel, delayed
from timeit import default_timer as timer

In [24]:
cp_file = '../../data/causal-priors.txt'
sde_file = '../../data/mouse_to_human_normalized.tsv'

cpo_df = pd.read_csv(cp_file, sep='\t', header=None, usecols=[0, 1, 2], names=['symbol', 'action', 'targetSymbol'])
cpo_df = cpo_df[cpo_df['action'].isin(['upregulates-expression', 'downregulates-expression'])]
cpo_df.reset_index(drop=True, inplace=True)
cpo_df['isUp'] = np.where(cpo_df['action'] == 'upregulates-expression', 1, -1)
cpo_df.drop(['action'], axis=1, inplace=True)

sde_df = pd.read_csv(sde_file, sep='\t', header=0, index_col=0).T
sde_df.replace(0, np.nan, inplace=True)
sde_df = pd.DataFrame(zscore(sde_df, nan_policy='omit'), index=sde_df.index, columns=sde_df.columns)
print("Files read complete...")

Files read complete...


In [25]:
# There may be some targetSymbols in cpo_df which are not present in sde_df
# Now remove those rows from cpo_df
cpo_df = cpo_df[cpo_df['targetSymbol'].isin(sde_df.columns)]
cpo_df.reset_index(drop=True, inplace=True)
cpo_df.head()

,symbol,targetSymbol,isUp
0,XBP1,TPP1,1
1,KLF5,CXCR4,1
2,ATF3,SELE,-1
3,MYC,EIF4G1,1
4,LDHA,HIF1A,1


In [26]:
cpo_grouped_df = cpo_df.groupby('symbol')['isUp'].apply(list).reset_index(name='upDownList')
cpo_grouped_df['targetList'] = cpo_df.groupby('symbol')['targetSymbol'].apply(list).reset_index(name='targetList')[
    'targetList']
cpo_grouped_df['upDownCount'] = cpo_grouped_df['upDownList'].apply(lambda x: len(x))
max_target = np.max(cpo_grouped_df['upDownCount'])
cpo_grouped_df.head()

,symbol,upDownList,targetList,upDownCount
0,A1BG,[-1],[A2M],1
1,A2M,[1],[STAT3],1
2,AATF,"[-1, -1]","[BAX, CTNNB1]",2
3,ABCA1,"[1, 1, -1, -1]","[NR1H2, NR1H3, GLI2, SREBF2]",4
4,ABCA3,"[1, 1, 1, 1, 1]","[SREBF1, GATA6, FOXA2, NFATC3, CEBPA]",5


### 1. Generating distribution using cumulative sum

In [27]:
# n = 10_000
# ranks = [(i + 0.5) / n for i in range(1, n)]
#
# # Generate Distribution
# iters = 10_000
#
# distribution = np.zeros((max_target, iters))
#
# # Took 13 seconds for 10,000 iterations
# for i in range(iters):
#     cs = np.random.choice(ranks, max_target, replace=False).cumsum()
#     for target in range(1, max_target + 1):
#         amr = cs[target - 1] / target
#         distribution[target - 1, i] = np.min([amr, 1 - amr])
#
# distribution = np.sort(distribution, axis=1)
# distribution
#
#
# # Took multiple minutes for 10,000 iterations
# # for target in range(1, max_target + 1):
# #     arr = np.zeros(iters)
# #     for i in range(iters):
# #         amr = np.mean(np.random.choice(ranks, target, replace=False))
# #         imr = 1 - amr
# #         arr[i] = np.min([amr, imr])
# #     distribution[target - 1] = np.sort(arr)
# #
# # distribution

In [28]:
# # Another way to generate distribution
# n = 10_000
# iters = 1000
# distribution = np.zeros((max_target, iters))
#
# for target in range(1, max_target + 1):
#     arr = np.zeros(iters)
#     for i in range(iters):
#         amr = (np.mean(np.random.choice(n, target, replace=False)) + 0.5) / n
#         imr = 1 - amr
#         arr[i] = np.min([amr, imr])
#     distribution[target - 1] = arr
#
# distribution

In [29]:
# # Another way to generate distribution
# n = 10_000
# distribution = []
# iters = 1000
# max_target = 218
#
# # Took 21 seconds to run
# for target in range(1, max_target + 1):
#     arr = []
#     for i in range(iters):
#         amr = (np.mean(np.random.choice(n, target, replace=False)) + 0.5) / n
#         imr = 1 - amr
#         arr.append(np.min([amr, imr]))
#     distribution.append(arr)
#
# distribution = np.array(distribution)
# distribution

### 2. Generating Distribution Using parallel processing

In [30]:
# n = 10_000
# ranks = [(i + 0.5) / n for i in range(1, n)]
# iters = 10_000
#
#
# # Worker function that each process will run
# # Took 1 min 46 seconds for 10,000 iterations
# def worker(target):
#     arr = np.zeros(iters)
#     for i in range(iters):
#         amr = np.mean(np.random.choice(ranks, target, replace=False))
#         imr = 1 - amr
#         arr[i] = np.min([amr, imr])
#     return np.sort(arr)
#
#
# distribution = Parallel(n_jobs=-1)(delayed(worker)(target) for target in range(1, max_target + 1))
#
# distribution = np.array(distribution)
# distribution

### 3. Can we combine both methods? Let's try

In [31]:
n = 10_000
ranks = [(i + 0.5) / n for i in range(1, n)]
iters = 1_000_000


def worker(iter):
    arr = np.zeros(max_target)
    cs = np.random.choice(ranks, max_target, replace=False).cumsum()
    for target in range(1, max_target + 1):
        amr = cs[target - 1] / target
        arr[target - 1] = np.min([amr, 1 - amr])
    return arr


distribution = Parallel(n_jobs=-1)(delayed(worker)(iter) for iter in range(iters))
# 1M x 218

# Transpose distribution
distribution = np.array(distribution).T
distribution = np.sort(distribution, axis=1)
distribution

KeyboardInterrupt: 

In [ ]:
# Save distribution
# np.save('data/distribution.npy', distribution)

# save to compressed npy file
np.savez_compressed('data/distribution.npz', distribution=distribution)

In [32]:
# load npz file to numpy array
distribution = np.load('data/distribution.npz')['distribution']
distribution

array([[5.00000000e-05, 5.00000000e-05, 5.00000000e-05, ...,
        4.99950000e-01, 4.99950000e-01, 4.99950000e-01],
       [4.50000000e-04, 5.00000000e-04, 6.00000000e-04, ...,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.95000000e-03, 7.98333333e-03, 8.38333333e-03, ...,
        4.99983333e-01, 4.99983333e-01, 4.99983333e-01],
       ...,
       [4.01414815e-01, 4.01656944e-01, 4.07365278e-01, ...,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [3.99960599e-01, 4.03874885e-01, 4.09326037e-01, ...,
        4.99999770e-01, 4.99999770e-01, 4.99999770e-01],
       [3.99909174e-01, 4.03860550e-01, 4.08567890e-01, ...,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01]])

In [ ]:
cpo_df_c_grouped = None
iters = 1_000_000
# Now Calculate actual RM for each cell

result = pd.DataFrame(columns=cpo_grouped_df['symbol'].values, index=sde_df.index)
cell_count = 0

for idx, row in sde_df.iterrows():
    time = timer()
    cell = pd.DataFrame(row.index, columns=['symbol'])
    cell['zscore'] = row.values
    # Remove rows of cell if zscore is nan
    cell.dropna(inplace=True)
    cell.reset_index(drop=True, inplace=True)
    cell.sort_values(by=['zscore'], ascending=[False], inplace=True)
    cell.reset_index(drop=True, inplace=True)

    cell['acti_rank'] = cell.index
    cell['acti_rank'] = (cell['acti_rank'] + 0.5) / len(cell)

    cell['inhi_rank'] = 1 - cell['acti_rank']

    cpo_df_c = cpo_df.copy()
    cpo_df_c = cpo_df_c[cpo_df_c['targetSymbol'].isin(cell['symbol'])]
    cpo_df_c.reset_index(drop=True, inplace=True)

    cpo_df_c['pos_rs'] = cpo_df_c.apply(
        lambda x:
        cell.loc[cell['symbol'] == x['targetSymbol'], 'acti_rank'].values[0]
        if x['isUp'] == 1
        else
        cell.loc[cell['symbol'] == x['targetSymbol'], 'inhi_rank'].values[0],
        axis=1
    )
    cpo_df_c['neg_rs'] = 1 - cpo_df_c['pos_rs']

    # Group
    cpo_df_c_grouped = cpo_df_c.groupby('symbol')['isUp'].apply(list).reset_index(name='upDownList')
    cpo_df_c_grouped['targetList'] = cpo_df_c.groupby('symbol')['targetSymbol'].apply(list).reset_index(
        name='targetList')['targetList']
    cpo_df_c_grouped['upDownCount'] = cpo_df_c_grouped['upDownList'].apply(lambda x: len(x))
    cpo_df_c_grouped['pos_rs'] = cpo_df_c.groupby('symbol')['pos_rs'].apply(list).reset_index(name='pos_rs')[
        'pos_rs']
    cpo_df_c_grouped['neg_rs'] = cpo_df_c.groupby('symbol')['neg_rs'].apply(list).reset_index(name='neg_rs')[
        'neg_rs']
    cpo_df_c_grouped['rs'] = cpo_df_c_grouped.apply(
        lambda x: np.min([np.mean(x['pos_rs']), np.mean(x['neg_rs'])]),
        axis=1
    )
    # if rs is pos_rs then it is up, if rs is neg_rs then it is down. So if rs is neg_rs then we need to multiply it by -1
    cpo_df_c_grouped['rs'] = cpo_df_c_grouped.apply(
        lambda x: x['rs'] if np.mean(x['pos_rs']) > np.mean(x['neg_rs']) else -1 * x['rs'],
        axis=1
    )

    # Counting how many times the RS is less than the random distribution
    for idx1, row1 in cpo_df_c_grouped.iterrows():
        target = row1['upDownCount']
        rs = row1['rs']
        arr = distribution[target - 1]
        count = np.searchsorted(arr, rs)
        if count == 0:
            count = count + 1
        if np.sign(rs) == -1:
            count = -1 * count
        cpo_df_c_grouped.loc[idx1, 'count'] = count
        cpo_df_c_grouped.loc[idx1, 'p-value'] = count / iters

    # Collect p-values of all cells
    result.loc[idx, cpo_df_c_grouped['symbol']] = cpo_df_c_grouped['p-value'].values
    cell_count += 1
    print(f"Cell number: {cell_count} Time taken:  {timer() - time} seconds")

# Remove columns if all values are nan
result.dropna(axis=1, how='all', inplace=True)
result.to_csv('data/result.tsv', sep='\t')

Cell number: 1 Time taken:  0.25178816200059373 seconds
Cell number: 2 Time taken:  0.3638568260066677 seconds
Cell number: 3 Time taken:  0.7845644510089187 seconds
Cell number: 4 Time taken:  0.5785783100000117 seconds
Cell number: 5 Time taken:  0.38797912500740495 seconds
Cell number: 6 Time taken:  0.3552623459981987 seconds
Cell number: 7 Time taken:  0.3884594949922757 seconds
Cell number: 8 Time taken:  0.28830916099832393 seconds
Cell number: 9 Time taken:  0.49517674300295766 seconds
Cell number: 10 Time taken:  0.6513310070004081 seconds
Cell number: 11 Time taken:  0.32213386200601235 seconds
Cell number: 12 Time taken:  0.4451601210021181 seconds
Cell number: 13 Time taken:  0.45942510399618186 seconds
Cell number: 14 Time taken:  0.19746850100636948 seconds
Cell number: 15 Time taken:  0.3327187300019432 seconds
Cell number: 16 Time taken:  0.39169119000143837 seconds
Cell number: 17 Time taken:  0.7579303250095109 seconds
Cell number: 18 Time taken:  0.46553524599585216 

In [ ]:
import matplotlib.pyplot as plt

read_dist = distribution.copy()
t1 = read_dist[5]

# bins and count from plt.hist
count, bins, ignored = plt.hist(t1, bins=1000)
plt.show()

# Merge count and bins
count = pd.DataFrame(count, columns=['count'])
